In [ ]:
import requests
import pandas as pd
import datetime
import json
import sqlite3
import traceback

# === 設定 ===
# DB_FILE = "/Users/lulutsai/Documents/NTPU_class/paper/code/mobile01_clawler/ntpu_paper.sqlite"
LOG_FILE = "eb_evaluation.log"
DB_FILE = "D:/NTPU_class/paper/code/mobile01_clawler/ntpu_paper.sqlite"
API_TOKEN = 'sk-a94e722134454665b8fef72ddb349f37'
API_URL = "http://localhost:3000/api/v1/chat/completions"
LIMIT_COUNT = 10 # ← 你想一次處理幾筆資料（可自行調整）
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_TOKEN}"
}

# === 🔹 Log 工具 ===
def write_log(message: str, link_id=None, article_id=None, comment_id=None, status=None):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    id_info = []
    if link_id:
        id_info.append(f"link_id={link_id}")
    if article_id:
        id_info.append(f"article_id={article_id}")
    if comment_id:
        id_info.append(f"comment_id={comment_id}")
    if status:
        id_info.append(f"status={status}")
    id_str = " | ".join(id_info)
    line = f"[{timestamp}] {message}"
    if id_str:
        line += f" | {id_str}"
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(line + "\n")
    print(line)


# === 🔹 確保欄位存在 ===
def ensure_column(conn, table, column, definition):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table});")
    columns = [c[1] for c in cur.fetchall()]
    if column not in columns:
        cur.execute(f"ALTER TABLE {table} ADD COLUMN {column} {definition};")
        conn.commit()
        write_log(f"🧱 已新增欄位 {table}.{column}")
    else:
        write_log(f"🔎 欄位 {column} 已存在，略過")


# === 🔹 建立 eb_evaluation 資料表 ===
def ensure_eb_evaluation_table(conn):
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS eb_evaluation (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        link_id TEXT NOT NULL,
        article_id TEXT NOT NULL,
        comment_id TEXT,
        level INTEGER DEFAULT 1,
        created_at TEXT,
        strategy_power REAL,
        strategy_emotion REAL,
        strategy_blame REAL,
        score_fear REAL,
        score_obligation REAL,
        score_guilt REAL,
        pua_source TEXT,
        main_strategy TEXT,
        main_strategy_detail TEXT,
        confidence REAL,
        score_overall REAL,
        model_name TEXT,
        model_version TEXT,
        knowledge_base TEXT,
        FOREIGN KEY (article_id) REFERENCES articles(id),
        FOREIGN KEY (link_id) REFERENCES links(id)
    );
    """)
    conn.commit()
    write_log("✅ 確認資料表 eb_evaluation 已存在")


# === 🔹 取得待分析文章 ===
def get_pending_articles(conn, limit):
    query = f"""
    SELECT id, link_id, content
    FROM articles
    WHERE content IS NOT NULL
      AND TRIM(content) != ''
      AND (pua_status IS NULL OR pua_status = 'pending')
    LIMIT {limit};
    """
    df = pd.read_sql_query(query, conn)
    write_log(f"📘 共讀取 {len(df)} 筆文章待分析（上限 {limit}）")
    return df


# === 🔹 更新文章狀態 ===
def update_article_status(conn, article_id, status, reason=None):
    cur = conn.cursor()
    cur.execute(
        "UPDATE articles SET pua_status = ?, pua_status_reason = ? WHERE id = ?;",
        (status, reason, article_id)
    )
    conn.commit()


# === 🔹 呼叫模型分析 ===
def analyze_text(text):
    data = {
        "model": "mistral:latest",
        "knowledge": ["pua_db"],
        "messages": [
            {
                "role": "system",
                "content": """你是一個以繁體中文回覆的情緒勒索分析助手。
請根據 Forward (1997) 與 陳燕諭 (2023) 對勒索策略的理論分類，
對給定文字進行情緒勒索分層評估，並輸出以下 JSON 結構：

{
  "strategy_power": 0~5,
  "strategy_emotion": 0~5,
  "strategy_blame": 0~5,
  "score_fear": 0~5,
  "score_obligation": 0~5,
  "score_guilt": 0~5,
  "pua_source": "family／partner／friend／workplace／online／self",
  "main_strategy": "power／emotion／blame／none",
  "main_strategy_detail": "更細緻的中文描述，例如『以哭訴方式威脅分手』或『混合型情緒操控』",
  "confidence": 0~1,
  "score_overall": 0~5
}

定義說明：
- 0 代表該特徵完全沒有出現；
- 5 代表該特徵非常明顯；
- main_strategy 僅能是 power、emotion、blame、none 四類；
- 所有分數均為數值，請勿包含文字說明；
- 請務必只輸出純 JSON，不要任何解釋文字。
"""
            },
            {"role": "user", "content": f"請分析這段文字: {text}"}
        ]
    }

    res = requests.post(API_URL, headers=headers, json=data, timeout=90)
    res.raise_for_status()
    content = res.json()["choices"][0]["message"]["content"]

    try:
        result = json.loads(content)
    except json.JSONDecodeError:
        import re
        match = re.search(r'\{.*\}', content, re.S)
        if match:
            result = json.loads(match.group())
        else:
            raise ValueError(f"無法解析 JSON：{content}")
    return result


# === 🔹 分數檢查 ===
def validate_scores(result, link_id, article_id):
    for key in ["strategy_power", "strategy_emotion", "strategy_blame",
                "score_fear", "score_obligation", "score_guilt", "score_overall"]:
        if key in result:
            val = result[key]
            try:
                if not (0 <= float(val) <= 5):
                    write_log(f"⚠️ {key} 超出範圍 ({val})，已設為 None",
                              link_id=link_id, article_id=article_id)
                    result[key] = None
            except Exception:
                result[key] = None
    if "confidence" in result:
        try:
            if not (0 <= float(result["confidence"]) <= 1):
                write_log(f"⚠️ confidence 超出範圍 ({result['confidence']})，已設為 None",
                          link_id=link_id, article_id=article_id)
                result["confidence"] = None
        except Exception:
            result["confidence"] = None
    return result


# === 🔹 寫入分析結果 ===
def insert_analysis_result(conn, link_id, article_id, result):
    cur = conn.cursor()
    created_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    result = validate_scores(result, link_id, article_id)

    cur.execute("""
        INSERT INTO eb_evaluation (
            link_id, article_id, comment_id, level, created_at,
            strategy_power, strategy_emotion, strategy_blame,
            score_fear, score_obligation, score_guilt,
            pua_source, main_strategy, main_strategy_detail,
            confidence, score_overall,
            model_name, model_version, knowledge_base
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """, (
        link_id, article_id, result.get("comment_id"),
        result.get("level", 1), created_at,
        result.get("strategy_power"),
        result.get("strategy_emotion"),
        result.get("strategy_blame"),
        result.get("score_fear"),
        result.get("score_obligation"),
        result.get("score_guilt"),
        result.get("pua_source"),
        result.get("main_strategy"),
        result.get("main_strategy_detail") or "無明確描述",
        result.get("confidence"),
        result.get("score_overall"),
        result.get("model_name", "mistral"),
        result.get("model_version", "latest"),
        result.get("knowledge_base", "pua_db")
    ))
    conn.commit()


# === 🔸 主程式 ===
def main():
    conn = sqlite3.connect(DB_FILE)
    ensure_column(conn, "articles", "pua_status", "TEXT DEFAULT 'pending'")
    ensure_column(conn, "articles", "pua_status_reason", "TEXT")
    ensure_eb_evaluation_table(conn)
    ensure_column(conn, "eb_evaluation", "main_strategy_detail", "TEXT")

    df = get_pending_articles(conn, LIMIT_COUNT)
    if df.empty:
        write_log("⚠️ 沒有待分析資料。")
        conn.close()
        return

    for idx, row in df.iterrows():
        link_id = row["link_id"]
        article_id = row["id"]
        text = row["content"]

        try:
            result = analyze_text(text)
            insert_analysis_result(conn, link_id, article_id, result)
            update_article_status(conn, article_id, "done")
            write_log(
                f"✅ DONE 第 {idx+1}/{len(df)} 筆 | overall={result.get('score_overall')} "
                f"| main={result.get('main_strategy')} | detail={result.get('main_strategy_detail')}",
                link_id=link_id, article_id=article_id, status="done"
            )
        except Exception as e:
            reason = f"{type(e).__name__}: {str(e)[:100]}"
            update_article_status(conn, article_id, "error", reason)
            write_log(f"❌ ERROR 第 {idx+1}/{len(df)} 筆 | {reason}",
                      link_id=link_id, article_id=article_id, status="error")
            traceback.print_exc()

    conn.close()
    write_log("🎉 全部完成！")


# === 執行 ===
if __name__ == "__main__":
    main()

[2025-11-12 15:43:39] 🔎 欄位 pua_status 已存在，略過
[2025-11-12 15:43:39] 🔎 欄位 pua_status_reason 已存在，略過
[2025-11-12 15:43:39] ✅ 確認資料表 eb_evaluation 已存在
[2025-11-12 15:43:39] 🔎 欄位 main_strategy_detail 已存在，略過
[2025-11-12 15:43:39] 📘 共讀取 10 筆文章待分析（上限 10）
[2025-11-12 15:43:43] ✅ DONE 第 1/10 筆 | overall=4.6 | main=power | detail=使用情緒勒索以控制子女，包括使用「感謝我們養你」、「為你好」等擔心和感激之語作為控制手段 | link_id=68f87edcaf1137700cb26e94 | article_id=68fce9b6af1137205015fd06 | status=done
[2025-11-12 15:43:47] ✅ DONE 第 2/10 筆 | overall=4.2 | main=emotion | detail=利用憤怒與責任感控制對方，以及使用電子紀錄和其他偵查方式增加壓力 | link_id=68f87edcaf1137700cb26e95 | article_id=68fce9f3af11377624f11eda | status=done
[2025-11-12 15:43:50] ✅ DONE 第 3/10 筆 | overall=3 | main=emotion | detail=使用反駁、負面描述和矛盾的語言來衝壓年輕人，並將忠告轉變為情緒勒索 | link_id=68f87edcaf1137700cb26e96 | article_id=68fcea01af11377624f11edb | status=done
[2025-11-12 15:43:54] ✅ DONE 第 4/10 筆 | overall=3.4 | main=emotion | detail=使用自我責任和情感引動方式控制情緒 | link_id=68f87edcaf1137700cb26e98 | article_id=68fcea15af113776